In [1]:
import os
import random
import numpy as np
import pandas as pd
import collections
import math
import nltk

In [2]:
uri_train  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'

df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

other_df = pd.DataFrame({"label": ['true', "false"]})

df_linear = df_train[df_train.label.isin(other_df.label)]

df_linear_1= df_linear.reset_index()
df_linear_1['statement'] = df_linear_1['statement'].str.lower()
#df_linear_1['statement']= df_linear_1['statement'].str.split()

In [3]:
uri_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

df_test = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

other_df_1 = pd.DataFrame({"label": ['true', "false"]})

df_linear_test = df_test[df_test.label.isin(other_df_1.label)]

df_linear_test_1= df_linear_test.reset_index()
df_linear_test_1['statement'] = df_linear_test_1['statement'].str.lower()

In [4]:
df_linear_1['tokenized_statement'] = df_linear_1.apply(lambda row: nltk.word_tokenize(row['statement']), axis=1)


In [5]:
#build a vocab 

#all the unique words in a dataset 
word_set = set()

for idx, row in df_linear_1.iterrows():
    word_set.update(row['tokenized_statement'])


In [6]:
#this is a look up table 
vocab = {word:idx for idx,word in enumerate(word_set)}

In [7]:
number_of_zeros_needed = len(vocab)


In [8]:
def vector_generator(sequence_toekns, vocab): 

    return_vector= np.zeros(number_of_zeros_needed)

    #set some positions within that vector 
    for token in  sequence_toekns: 
        pos = vocab.get(token)
        if pos is not None:
            return_vector[pos] +=1



    return(return_vector)


In [19]:
def avg_perceptron(df_train, df_test):

    count = 0

    #weights for the model 
    theta_true= np.zeros(number_of_zeros_needed)
    theta_false= np.zeros(number_of_zeros_needed)

    avg_weight_true=np.zeros(number_of_zeros_needed) 
    avg_weight_false=np.zeros(number_of_zeros_needed) 


    while count<100:

        df_train_sample= df_train.sample()

        #get us the lable 
        actual_label = df_train_sample["label"].iat[0]

        #one hot vector for sentence 
        x_vector= vector_generator(df_train_sample["tokenized_statement"].iat[0],vocab)

        #To James- why the .dot operation?? 

        #for i in range(len(theta)):

        #calc scores for algo 
        theta_true_in_loop = np.dot(x_vector,theta_true)
        theta_false_in_loop= np.dot(x_vector,theta_false)

        predicted_label = "true" if theta_true_in_loop >= theta_false_in_loop else "false"

        if actual_label != predicted_label:

            print("wrong")

            #To James- need more explanation on this part 
            if actual_label == "false":
                
                #34:08 
                #penalize if mistake and encourage if right 
                #we want theta false to go higher 

                theta_false = theta_false +x_vector #I want the actual label to have more weight on those words 
                theta_true = theta_true -x_vector 
            else: 
                theta_false = theta_false -x_vector 
                theta_true = theta_true +x_vector 
        else:
            print("right")

        avg_weight_true += theta_true
        avg_weight_false+= theta_false

        
        count+=1 


    return avg_weight_true/count, avg_weight_false/count

In [20]:
avg_weight_true,avg_weight_false= avg_perceptron(df_linear_1, df_linear_test_1)

right
wrong
wrong
wrong
right
right
wrong
wrong
right
right
right
wrong
wrong
wrong
right
wrong
right
right
right
wrong
right
right
right
wrong
wrong
wrong
right
wrong
right
right
right
wrong
right
wrong
wrong
right
wrong
wrong
right
right
wrong
right
wrong
wrong
wrong
wrong
wrong
right
wrong
right
right
right
right
right
wrong
right
wrong
wrong
right
right
wrong
right
right
right
wrong
right
wrong
wrong
right
wrong
right
right
right
right
right
right
wrong
wrong
wrong
right
right
wrong
wrong
right
right
wrong
right
right
wrong
wrong
right
wrong
wrong
right
wrong
wrong
right
right
right
wrong


In [22]:
print(np.nonzero(avg_weight_true)) #these are indexes 

(array([   5,   16,   22,   32,  103,  119,  122,  124,  132,  136,  143,
        146,  177,  194,  204,  222,  237,  241,  257,  272,  274,  287,
        297,  359,  367,  377,  409,  436,  465,  478,  488,  522,  543,
        556,  557,  570,  571,  586,  593,  599,  600,  621,  624,  672,
        689,  710,  713,  756,  763,  767,  769,  770,  774,  777,  797,
        799,  802,  807,  812,  861,  868,  913,  914,  925,  937,  967,
        975,  993, 1000, 1003, 1016, 1064, 1100, 1101, 1124, 1131, 1134,
       1146, 1201, 1209, 1224, 1272, 1273, 1279, 1295, 1314, 1317, 1347,
       1372, 1427, 1437, 1447, 1454, 1466, 1475, 1480, 1483, 1491, 1572,
       1576, 1624, 1636, 1644, 1645, 1647, 1668, 1688, 1750, 1755, 1793,
       1795, 1797, 1800, 1811, 1822, 1862, 1865, 1886, 1888, 1916, 1940,
       1941, 1952, 1974, 2018, 2020, 2049, 2052, 2063, 2113, 2121, 2186,
       2208, 2211, 2212, 2255, 2263, 2275, 2278, 2302, 2318, 2339, 2386,
       2409, 2426, 2435, 2453, 2463, 2495, 2513, 2

In [ ]:

from calendar import c


for i,row in df_test.iterrows():

    #get us the lable 
    actual_label = row["label"].iat[0]

    #one hot vector for sentence 
    x_vector= vector_generator(row["tokenized_statement"].iat[0],vocab)

    #To James- why the .dot operation?? 

    #for i in range(len(theta)):

    #calc scores for algo 
    theta_true_in_loop = np.dot(x_vector, avg_weight_true ) #averaged weight
    theta_false_in_loop= np.dot(x_vector, avg_weight_false)

    predicted_label = "true" if theta_true_in_loop >= theta_false_in_loop else "false"

    if actual_label != predicted_label:

        print("wrong")
    else:
        print("right")


    

